In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vgurramkonda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
import os
from openai import AzureOpenAI
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from IPython.display import HTML
import markdown
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import io
import ast
import datetime as dt
import math
import re
import statistics
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import nltk
from nltk.corpus import stopwords

# Load environment variables
load_dotenv("user_env.env", override=True)

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_CHAT_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE")
)

# Widgets
"""file_uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
prompt_input = widgets.Textarea(placeholder='Enter your question about the data...', layout=widgets.Layout(width='100%', height='80px'))
submit_button = widgets.Button(description="Submit", button_style='success')
chat_output = widgets.Output()
"""
# Global DataFrame
df = None

# Helper Functions
def get_unique_values_for_columns(df, columns):
    unique_map = {}
    for col in columns:
        unique_map[col] = sorted(df[col].dropna().unique().tolist())
    return unique_map

stop_words = set(stopwords.words("english"))
custom_words = {"client", "project", "summary", "want", "give", "best", "bank", "Services", "Business", "private", "limited"}
stop_words.update(custom_words)

def clean_query(query):
    global stop_words
    tokens = re.findall(r'\b\w+\b', query.lower())
    return [token for token in tokens if token not in stopwords]

def match_query_to_unique_values_formatted(query, unique_value_map):
    query_tokens = clean_query(query)
    output_lines = []

    for col, unique_values in unique_value_map.items():
        matched_values = []
        for val in unique_values:
            val_tokens = re.findall(r'\b\w+\b', val.lower())
            if any(qt == vt for qt in query_tokens for vt in val_tokens):
                matched_values.append(val)
        
        if matched_values:
            formatted = f"- {col}: {matched_values}"
            output_lines.append(formatted)

    return "\n".join(output_lines)

    
def extract_executable_formula(llm_response: str) -> str:
    """
    Cleans the LLM response to extract a plain, executable Python formula.
    Removes code fences and leading/trailing whitespace.
    """
    lines = llm_response.strip().splitlines()
    # Remove code fences and empty lines
    cleaned_lines = [line for line in lines if not line.strip().startswith("```")]
    return "\n".join(cleaned_lines).strip()

def update_chat_display():
    with chat_output_area:
        clear_output(wait=True)
        for user, bot in chat_history:
            display(format_message(user, 'user'))
            display(format_message(bot, 'bot'))


def evaluate_formula(df, formula_str):
    formula_str = formula_str.strip()
    try:
        return ast.literal_eval(formula_str)
    except (ValueError, SyntaxError):
        pass

    local_vars = {
        'df': df,
        'pd': pd,
        'np': np,
        'dt': dt,
        'datetime': dt,
        'math': math,
        're': re,
        'statistics': statistics,
        'len': len,
        'sum': sum,
        'min': min,
        'max': max,
        'abs': abs,
        'round': round,
        'range': range,
        'list': list,
        'dict': dict,
        'set': set,
        'sorted': sorted,
        'zip': zip,
        'enumerate': enumerate,
        'any': any,
        'all': all,
        'map': map,
        'filter': filter,
        'str': str,
        'int': int,
        'float': float,
        'bool': bool,
    }

    try:
        return eval(formula_str, {"__builtins__": {}}, local_vars)
    except Exception as e:
        return f"Error evaluating formula: {e}"



def generate_formula_with_llm(user_prompt, cat_cols_names, date_cols_names, num_cols_names, unique_value_map):
    
    unique_val_info = "\n".join([f"- {col}: {vals}" for col, vals in unique_value_map.items()])
    
    prompt = f"""
    you are a precise Python formula generator for pandas DataFrames.
    The user has requested the following: "{user_prompt}"
    Here is the structure of the Excel dataset:
    - **Categorical Columns**: {cat_cols_names}
    - **Date Columns**: {date_cols_names}
    - **Numerical Columns**: {num_cols_names}

    Additional Information:
    
    Your task is to:
    1. Understand the user's intent, even if the column names mentioned in the request are approximate or partial (e.g., "Company" may refer to "Company Name").
    2. Match the user's request to the most relevant column names and their unique values.
    3. Generate a formula that:
    - Accurately fulfills the user's request.
    - Uses appropriate columns (categorical, date, numerical).
    - Is syntactically correct and ready to use in Excel.
    4. Provide a brief explanation of what the formula does and how it works.
    Return only the Python code in "" quotes and code should only contain like this "df.loc[df['Industry'] == 'Consumer', 'Value (in Cr.)'].sum()" 
    5. df is the dataframe.
    """

    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
        messages=[
            {"role": "system", "content": "You are a precise python dataframe formula generator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    return response.choices[0].message.content.strip()
    
#Generate a cleaned response
def generate_cleaned_response_with_llm(cleaned_response):
    prompt = f"""
    -This is the formula used for dataframe a : "{cleaned_response}"
    -Your task is to modify this formula to a single formula
    - In simple words NO storing variables like "x = formula" or "df['y'] = formula" , It SHOULD be in format "formula1,formula2"
    - But also modify the formula such that no new Column (df['new_column']) needs to be added
    - DONT Provide any explanations
    """
    response=client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
        messages=[
            {"role": "system", "content": "You are a precise python dataframe formula generator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def create_user_response(result, user_prompt):
    prompt = f"""
    The user asked: "{user_prompt}"
    The result of the computation based on their request is: {result}
    Write a clear, friendly, and concise response that:
    - Acknowledges the user's request.
    - Presents the result in a helpful and natural way.
    - Avoids technical jargon unless necessary.
    - Sounds like a helpful assistant explaining the outcome.
    Only return the final response to the user.
    """

    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# Main interaction logic
# Widgets
'''
header = widgets.HTML(value="""
<div style="
    color: #8cc751;
    font-size: 26px;
    font-weight: bold;
    text-align: left;
    margin-bottom: 20px;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
">
     Excel ChatBot
</div>
""")'''

from IPython.display import HTML

display(HTML("""
<style>
.chat-table {
    width: 100%;
    border-collapse: collapse;
}
.chat-table th, .chat-table td {
    border: 1px solid #ccc;
    padding: 6px 10px;
    text-align: left;
}
.chat-table th {
    background-color: #2e7d32;
    color: white;
}
</style>
"""))


#Widgets
file_uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
prompt_input = widgets.Textarea(placeholder='Ask a question about your data...', layout=widgets.Layout(width='100%', height='80px'))
submit_button = widgets.Button(description="Submit", button_style='success')
clear_button = widgets.Button(description="Clear Chat", button_style='danger')
chat_output_area = widgets.Output()
chat_scroll_box = widgets.VBox([chat_output_area], layout=widgets.Layout(
    border='1px solid #ccc',
    height='400px',
    overflow_y='auto',
    padding='10px'
))

# Global state
df = None
chat_history = []

# Disable chat input and submit button initially
prompt_input.disabled = True
submit_button.disabled = True

def format_message(message, sender='user'):
    color = '#d4edda' if sender == 'user' else '#155724'
    text_color = 'black' if sender == 'user' else 'white'
    align = 'flex-start' if sender == 'user' else 'flex-end'

    # Detect Markdown table and convert to HTML
    if isinstance(message, str) and re.search(r"\|.+\|", message):
        message_html = markdown.markdown(message, extensions=['tables'])
    elif isinstance(message, pd.DataFrame):
        message_html = message.to_html(index=False, border=0)
    else:
        message = re.sub(r'\*\*(.*?)\*\*', r'\1', message)
        message_html = f"<div>{message}</div>"

    bubble = widgets.HTML(
        value=f"""
        <div style='padding:10px; border-radius:10px; background-color:{color}; color:{text_color}; word-wrap: break-word;'>
            {message_html}
        </div>
        """,
        layout=widgets.Layout(width='auto')
    )

    return widgets.HBox([bubble], layout=widgets.Layout(justify_content=align, width='100%', padding='5px 0'))


# Update chat display
'''
def update_chat_display():
    chat_scroll_box.children = [
        format_message(user, 'user') if i % 2 == 0 else format_message(bot, 'bot')
        for i, (user, bot) in enumerate(chat_history)
    ]
    '''

# File upload handler
def on_file_upload(change):
    global df
    if file_uploader.value:
        uploaded_file = list(file_uploader.value)[0]
        df = pd.read_excel(io.BytesIO(uploaded_file['content']), sheet_name='Sheet1')

        # Enable chat input and submit button
        prompt_input.disabled = False
        submit_button.disabled = False

        with chat_output_area:
            clear_output()
            print("File uploaded successfully. You can now ask questions.")

# Attach observer
file_uploader.observe(on_file_upload, names='value')

# Submit button handler
def on_submit_clicked(b):
    global df
    if df is None:
        return

    user_prompt = prompt_input.value.strip()
    if not user_prompt:
        return

    # Add user message and "Thinking..." placeholder
    chat_history.append((user_prompt, "Thinking..."))
    update_chat_display()

    # Process data
    num_cols = df.select_dtypes(include=['number'])
    cat_cols = df.select_dtypes(exclude=['number', 'datetime'])
    date_cols = df.select_dtypes(include=['datetime'])
    unique_value_map = get_unique_values_for_columns(df, ['Stage', 'New L2', 'New L3', 'Business Level 1'])
    #EngAcc_unique_value_map = get_unique_values_for_columns(df, ['English Account Name'])
    #EngAcc_matches = match_query_to_unique_values(query, EngAcc_unique_value_map)
    

    response_llm = generate_formula_with_llm(user_prompt, cat_cols.columns.tolist(), date_cols.columns.tolist(), num_cols.columns.tolist(), unique_value_map)
    cleaned_response = extract_executable_formula(generate_cleaned_response_with_llm(response_llm))
    result = evaluate_formula(df, cleaned_response)
    friendly_response = create_user_response(result, user_prompt)

    # Replace "Thinking..." with actual response
    chat_history[-1] = (user_prompt, friendly_response)
    update_chat_display()
    prompt_input.value = ""

# Clear button handler
def on_clear_clicked(b):
    global chat_history
    chat_output_area.clear_output()
    chat_history = []
    update_chat_display()

# Bind buttons
submit_button.on_click(on_submit_clicked)
clear_button.on_click(on_clear_clicked)

# Display UI
display(widgets.VBox([
    #header,
    widgets.Label("Upload your Excel file:"),
    file_uploader,
    chat_scroll_box,
    prompt_input,
    widgets.HBox([submit_button, clear_button])
    #chat_output_area
]))



